# Pandemic financial analysis

In [1]:
# Import warnings + watermark
from watermark import watermark
from warnings import filterwarnings
filterwarnings("ignore")
print(watermark())
print(watermark(iversions=True, globals_=globals()))

Last updated: 2022-11-19T21:29:16.408979-08:00

Python implementation: CPython
Python version       : 3.7.13
IPython version      : 7.31.1

Compiler    : Clang 12.0.0 
OS          : Darwin
Release     : 22.1.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit




In [2]:
#import the necessary modules
import pandas as pd
import numpy as np
#questionary
from pathlib import Path
import fire
import questionary
#API
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline

In [3]:
POLYGON_API_KEY ='enZp2AUpH4pGXJJuQ1CbjdVXJIsBFBEl'

In [4]:
ticker_list = ["AMZN", "RTH", "AMT", "IYR", "XOM", "XLE", "SPY"]

In [5]:
import requests
import datetime
from time import sleep
from tqdm import tqdm
def get_prices(start_date,end_date,universe):
    data = pd.DataFrame()
    # end_at = datetime.datetime.now() 
    # begin_from = end_at + datetime.timedelta(days=-5000)
    for t in tqdm(universe):
        print(t)
        print(len(data))
        # for start in pd.date_range(start = begin_from, end = end_at,normalize=True,freq = '88D'):
        #     end = start + datetime.timedelta(days = 88)
        #     end = str(int((end - datetime.datetime(1970,1,1)).total_seconds()))
        #     start = str(int((start - datetime.datetime(1970,1,1)).total_seconds()))
        try:
            r = requests.get(f'https://api.polygon.io/v2/aggs/ticker/{t}/range/1/day/{start_date}/{end_date}?apiKey={POLYGON_API_KEY}')
        except json.JSONDecodeError:
            pass
        try:
            # print(r.json())
            temp_data = pd.DataFrame(r.json()['results'])
            temp_data['ticker'] = t
            data = data.append(temp_data, ignore_index=True)
            print(t+':'+str(pd.to_datetime(start_date, unit='s'))+':'+str(pd.to_datetime(end_date, unit='s')))
        except KeyError:
            print(f'{t} was not found')
            pass
        except ValueError:
            pass
        except ConnectionError as error:
            print(error)
            sleep(200)
            continue
        except TimeoutError as error:
            print(error)
            sleep(200)
            continue
        sleep(0.25)
#     data.to_csv(file_name,index = False)
    data.t = pd.to_datetime(data.t, unit = 'ms')
    columns_name = {'t': 'time', 'o': 'Open', 'c': 'Close', 'h': 'High', 'l': 'Low', 'v': 'Volume'} 
    data= data.rename(columns = columns_name)
    data = data[['time','ticker','Open', 'Close','High', 'Low', 'Volume']]
    data = data.set_index(['time', 'ticker'])
    data["daily_returns"] = data['Close'].groupby('ticker').pct_change()
    data = data.dropna()
    #data.to_csv(file_name, index = False)
    return data

In [6]:
# Create `start_date` and `end_date` variables for the period before the pandemic
start_date = "2020-03-01"
end_date = "2022-03-01"
all_data = get_prices(start_date=start_date, end_date=end_date, universe=ticker_list)


  0%|                                                                                                                | 0/7 [00:00<?, ?it/s]

AMZN
0


 14%|██████████████▊                                                                                         | 1/7 [00:00<00:03,  1.56it/s]

RTH
505


 29%|█████████████████████████████▋                                                                          | 2/7 [00:01<00:03,  1.61it/s]

...

 71%|██████████████████████████████████████████████████████████████████████████▎                             | 5/7 [00:03<00:01,  1.52it/s]

XLE
2525


 86%|█████████████████████████████████████████████████████████████████████████████████████████▏              | 6/7 [00:04<00:00,  1.42it/s]

SPY
3030


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.46it/s]


In [7]:
all_data


,,Open,Close,High,Low,Volume,daily_returns
time,ticker,,,,,,
2020-03-03 05:00:00,AMZN,98.7685,95.4495,99.8165,94.4045,150689820.0,-0.023010
2020-03-04 05:00:00,AMZN,97.3285,98.7915,98.9000,96.1000,95289620.0,0.035013
2020-03-05 05:00:00,AMZN,96.6500,96.2015,98.0360,95.5000,94964200.0,-0.026217
2020-03-06 05:00:00,AMZN,93.7500,95.0545,95.5435,93.4750,105471600.0,-0.011923
2020-03-09 04:00:00,AMZN,88.6930,90.0305,93.1385,88.0645,156264640.0,-0.052854
...,...,...,...,...,...,...,...
2022-02-23 05:00:00,SPY,432.6600,421.9500,433.2600,421.3500,132374005.0,-0.017739
2022-02-24 05:00:00,SPY,411.0200,428.3000,428.7600,410.6400,213923446.0,0.015049
2022-02-25 05:00:00,SPY,429.6100,437.7500,437.8400,427.8600,121785659.0,0.022064


In [8]:
all_data.index

MultiIndex([('2020-03-03 05:00:00', 'AMZN'),
            ('2020-03-04 05:00:00', 'AMZN'),
            ('2020-03-05 05:00:00', 'AMZN'),
            ('2020-03-06 05:00:00', 'AMZN'),
            ('2020-03-09 04:00:00', 'AMZN'),
            ('2020-03-10 04:00:00', 'AMZN'),
            ('2020-03-11 04:00:00', 'AMZN'),
            ('2020-03-12 04:00:00', 'AMZN'),
            ('2020-03-13 04:00:00', 'AMZN'),
            ('2020-03-16 04:00:00', 'AMZN'),
            ...
            ('2022-02-15 05:00:00',  'SPY'),
            ('2022-02-16 05:00:00',  'SPY'),
            ('2022-02-17 05:00:00',  'SPY'),
            ('2022-02-18 05:00:00',  'SPY'),
            ('2022-02-22 05:00:00',  'SPY'),
            ('2022-02-23 05:00:00',  'SPY'),
            ('2022-02-24 05:00:00',  'SPY'),
            ('2022-02-25 05:00:00',  'SPY'),
            ('2022-02-28 05:00:00',  'SPY'),
            ('2022-03-01 05:00:00',  'SPY')],
           names=['time', 'ticker'], length=3528)

In [9]:
idx = pd.IndexSlice

In [10]:
AMZN_daily_returns_df = all_data.loc[idx[:,'AMZN'],['daily_returns']]
AMZN_daily_returns_df

,,daily_returns
time,ticker,
2020-03-03 05:00:00,AMZN,-0.023010
2020-03-04 05:00:00,AMZN,0.035013
2020-03-05 05:00:00,AMZN,-0.026217
2020-03-06 05:00:00,AMZN,-0.011923
2020-03-09 04:00:00,AMZN,-0.052854
...,...,...
2022-02-23 05:00:00,AMZN,-0.035756
2022-02-24 05:00:00,AMZN,0.045095
2022-02-25 05:00:00,AMZN,0.016058


In [11]:
RTH_daily_returns_df = all_data.loc[idx[:,'RTH'],['daily_returns']]
RTH_daily_returns_df

,,daily_returns
time,ticker,
2020-03-03 05:00:00,RTH,-0.025206
2020-03-04 05:00:00,RTH,0.039796
2020-03-05 05:00:00,RTH,-0.022688
2020-03-06 05:00:00,RTH,-0.009098
2020-03-09 04:00:00,RTH,-0.042211
...,...,...
2022-02-23 05:00:00,RTH,-0.020849
2022-02-24 05:00:00,RTH,0.014314
2022-02-25 05:00:00,RTH,0.025366


In [12]:
AMT_daily_returns_df = all_data.loc[idx[:,'AMT'],['daily_returns']]
AMT_daily_returns_df

,,daily_returns
time,ticker,
2020-03-03 05:00:00,AMT,0.007892
2020-03-04 05:00:00,AMT,0.037132
2020-03-05 05:00:00,AMT,-0.018755
2020-03-06 05:00:00,AMT,-0.002511
2020-03-09 04:00:00,AMT,-0.058704
...,...,...
2022-02-23 05:00:00,AMT,-0.004029
2022-02-24 05:00:00,AMT,0.018905
2022-02-25 05:00:00,AMT,0.008242


In [13]:
IYR_daily_returns_df = all_data.loc[idx[:,'IYR'],['daily_returns']]
IYR_daily_returns_df

,,daily_returns
time,ticker,
2020-03-03 05:00:00,IYR,0.000109
2020-03-04 05:00:00,IYR,0.038201
2020-03-05 05:00:00,IYR,-0.021762
2020-03-06 05:00:00,IYR,-0.016980
2020-03-09 04:00:00,IYR,-0.074123
...,...,...
2022-02-23 05:00:00,IYR,-0.017626
2022-02-24 05:00:00,IYR,0.017740
2022-02-25 05:00:00,IYR,0.024067


In [14]:
XOM_daily_returns_df = all_data.loc[idx[:,'XOM'],['daily_returns']]
XOM_daily_returns_df

,,daily_returns
time,ticker,
2020-03-03 05:00:00,XOM,-0.047884
2020-03-04 05:00:00,XOM,0.021832
2020-03-05 05:00:00,XOM,-0.044067
2020-03-06 05:00:00,XOM,-0.048294
2020-03-09 04:00:00,XOM,-0.122248
...,...,...
2022-02-23 05:00:00,XOM,0.004054
2022-02-24 05:00:00,XOM,-0.012635
2022-02-25 05:00:00,XOM,0.026913


In [15]:
XLE_daily_returns_df = all_data.loc[idx[:,'XLE'],['daily_returns']]
XLE_daily_returns_df

,,daily_returns
time,ticker,
2020-03-03 05:00:00,XLE,-0.030573
2020-03-04 05:00:00,XLE,0.023653
2020-03-05 05:00:00,XLE,-0.036585
2020-03-06 05:00:00,XLE,-0.056185
2020-03-09 04:00:00,XLE,-0.201412
...,...,...
2022-02-23 05:00:00,XLE,0.010306
2022-02-24 05:00:00,XLE,-0.007836
2022-02-25 05:00:00,XLE,0.026375


In [16]:
SPY_daily_returns_df = all_data.loc[idx[:,'SPY'],['daily_returns']]
SPY_daily_returns_df

,,daily_returns
time,ticker,
2020-03-03 05:00:00,SPY,-0.028632
2020-03-04 05:00:00,SPY,0.042033
2020-03-05 05:00:00,SPY,-0.033242
2020-03-06 05:00:00,SPY,-0.016531
2020-03-09 04:00:00,SPY,-0.078095
...,...,...
2022-02-23 05:00:00,SPY,-0.017739
2022-02-24 05:00:00,SPY,0.015049
2022-02-25 05:00:00,SPY,0.022064


In [17]:
tech_daily_returns = pd.concat([AMZN_daily_returns_df, RTH_daily_returns_df], keys=["AMZN","RTH"])
tech_daily_returns

daily_returns
     time                ticker               
AMZN 2020-03-03 05:00:00 AMZN        -0.023010
     2020-03-04 05:00:00 AMZN         0.035013
     2020-03-05 05:00:00 AMZN        -0.026217
     2020-03-06 05:00:00 AMZN        -0.011923
     2020-03-09 04:00:00 AMZN        -0.052854
...                                        ...
RTH  2022-02-23 05:00:00 RTH         -0.020849
     2022-02-24 05:00:00 RTH          0.014314
     2022-02-25 05:00:00 RTH          0.025366
     2022-02-28 05:00:00 RTH         -0.002389
     2022-03-01 05:00:00 RTH          0.000684

[1008 rows x 1 columns]

In [18]:
RE_daily_returns = pd.concat([AMT_daily_returns_df, IYR_daily_returns_df], keys=["AMT","IYR"])
RE_daily_returns

daily_returns
    time                ticker               
AMT 2020-03-03 05:00:00 AMT          0.007892
    2020-03-04 05:00:00 AMT          0.037132
    2020-03-05 05:00:00 AMT         -0.018755
    2020-03-06 05:00:00 AMT         -0.002511
    2020-03-09 04:00:00 AMT         -0.058704
...                                       ...
IYR 2022-02-23 05:00:00 IYR         -0.017626
    2022-02-24 05:00:00 IYR          0.017740
    2022-02-25 05:00:00 IYR          0.024067
    2022-02-28 05:00:00 IYR         -0.016538
    2022-03-01 05:00:00 IYR         -0.002557

[1008 rows x 1 columns]

In [19]:
energy_daily_returns = pd.concat([XOM_daily_returns_df, XLE_daily_returns_df], keys=["XOM","XLE"])
energy_daily_returns

daily_returns
    time                ticker               
XOM 2020-03-03 05:00:00 XOM         -0.047884
    2020-03-04 05:00:00 XOM          0.021832
    2020-03-05 05:00:00 XOM         -0.044067
    2020-03-06 05:00:00 XOM         -0.048294
    2020-03-09 04:00:00 XOM         -0.122248
...                                       ...
XLE 2022-02-23 05:00:00 XLE          0.010306
    2022-02-24 05:00:00 XLE         -0.007836
    2022-02-25 05:00:00 XLE          0.026375
    2022-02-28 05:00:00 XLE          0.024681
    2022-03-01 05:00:00 XLE          0.010060

[1008 rows x 1 columns]

### A) Analyzing Each Stock and ETF Individually


In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
from pytz import timezone
#from IPython.display import clear_output
from time import sleep
import numpy as np
import pyfolio as pf
import sqlalchemy

In [59]:
pandemic_prices_df = pd.concat([AMZN_daily_returns_df.reset_index(level = 1,drop=True), RTH_daily_returns_df.reset_index(level = 1,drop=True), AMT_daily_returns_df.reset_index(level = 1,drop=True), IYR_daily_returns_df.reset_index(level = 1,drop=True), XOM_daily_returns_df.reset_index(level = 1,drop=True), XLE_daily_returns_df.reset_index(level = 1,drop=True), SPY_daily_returns_df.reset_index(level = 1,drop=True)],axis=0, keys=["AMZN","RTH","AMT","IYR","XOM","XLE","SPY"])
pandemic_prices_df

daily_returns
     time                              
AMZN 2020-03-03 05:00:00      -0.023010
     2020-03-04 05:00:00       0.035013
     2020-03-05 05:00:00      -0.026217
     2020-03-06 05:00:00      -0.011923
     2020-03-09 04:00:00      -0.052854
...                                 ...
SPY  2022-02-23 05:00:00      -0.017739
     2022-02-24 05:00:00       0.015049
     2022-02-25 05:00:00       0.022064
     2022-02-28 05:00:00      -0.002559
     2022-03-01 05:00:00      -0.015230

[3528 rows x 1 columns]

In [23]:
database_connection_string = 'sqlite:///'

In [24]:
engine = sqlalchemy.create_engine(database_connection_string)

In [25]:
engine.table_names()

[]

In [26]:
#Summary statistics for AMZN stock using .std() and .mean()
display(AMZN_daily_returns_df.std())
display(AMZN_daily_returns_df.mean())

daily_returns    0.021559
dtype: float64

daily_returns    0.001097
dtype: float64

In [27]:
#Summary statistics for RTH stock using .std() and .mean()
RTH_daily_returns_df.std()
RTH_daily_returns_df.mean()

daily_returns    0.000884
dtype: float64

In [28]:
#Summary statistics for AMT stock using .std() and .mean()
AMT_daily_returns_df.std()
AMT_daily_returns_df.mean()

daily_returns    0.000131
dtype: float64

In [29]:
#Summary statistics for IYR stock using .std() and .mean()
IYR_daily_returns_df.std()
IYR_daily_returns_df.mean()

daily_returns    0.000391
dtype: float64

In [30]:
#Summary statistics for XOM stock using .std() and .mean()
XOM_daily_returns_df.std()
XOM_daily_returns_df.mean()

daily_returns    0.001127
dtype: float64

In [31]:
#Summary statistics for XLE stock using .std() and .mean()
XLE_daily_returns_df.std()
XLE_daily_returns_df.mean()

daily_returns    0.001281
dtype: float64

In [32]:
pandemic_prices_df.describe()

,daily_returns
count,3528.000000
mean,0.000813
std,0.022020
min,-0.201412
25%,-0.008630
50%,0.000916
75%,0.010146
max,0.160373


In [33]:
#Summary statistics for AMZN using .describe()
AMZN_daily_returns_df.describe()

,daily_returns
count,504.000000
mean,0.001097
std,0.021559
min,-0.079221
25%,-0.010765
50%,0.001172
75%,0.011710
max,0.135359


In [34]:
#Summary statistics for RTH using .describe()
RTH_daily_returns_df.describe()

,daily_returns
count,504.000000
mean,0.000884
std,0.014332
min,-0.103722
25%,-0.004818
50%,0.001504
75%,0.007989
max,0.072946


In [35]:
#Summary statistics for AMT using .describe()
AMT_daily_returns_df.describe()

,daily_returns
count,504.000000
mean,0.000131
std,0.021656
min,-0.151638
25%,-0.009146
50%,-0.000023
75%,0.009113
max,0.122219


In [36]:
#Summary statistics for IYR using .describe()
IYR_daily_returns_df.describe()

,daily_returns
count,504.000000
mean,0.000391
std,0.019258
min,-0.168700
25%,-0.007200
50%,0.000915
75%,0.008129
max,0.085328


In [37]:
#Summary statistics for XOM using .describe()
XOM_daily_returns_df.describe()

,daily_returns
count,504.000000
mean,0.001127
std,0.026977
min,-0.122248
25%,-0.013158
50%,-0.000045
75%,0.014287
max,0.126868


In [38]:
#Summary statistics for XLE using .describe()
XLE_daily_returns_df.describe()

,daily_returns
count,504.000000
mean,0.001281
std,0.030101
min,-0.201412
25%,-0.012374
50%,0.000789
75%,0.015856
max,0.160373


In [39]:
# Variance Of Amzn
AMZN_var=AMZN_daily_returns_df.var()
AMZN_var

daily_returns    0.000465
dtype: float64

In [40]:
# Variance For RTH
RTH_var=RTH_daily_returns_df.var()
RTH_var

daily_returns    0.000205
dtype: float64

In [41]:
# Variance Of IYR
IYR_var=IYR_daily_returns_df.var()
IYR_var

daily_returns    0.000371
dtype: float64

In [42]:
# Variance Of AMT
AMT_var=AMT_daily_returns_df.var()
AMT_var

daily_returns    0.000469
dtype: float64

In [43]:
# Variance Of XOM
XOM_var=XOM_daily_returns_df.var()
XOM_var

daily_returns    0.000728
dtype: float64

In [44]:
# Variance Of XLE
XLE_var=XLE_daily_returns_df.var()
XLE_var

daily_returns    0.000906
dtype: float64

In [45]:
# Covariance for Each Stock and ETF
AMZN_Cov=AMZN_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
AMZN_Cov

0.00020195705481941884

In [46]:
RTH_Cov=RTH_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
RTH_Cov

0.00020693644628238053

In [47]:
# Covariance for Each Stock and ETF
AMT_Cov=AMT_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
AMT_Cov

0.0002448729063446677

In [48]:
IYR_Cov=IYR_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
IYR_Cov

0.0002640480626083232

In [49]:
XOM_Cov=XOM_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
XOM_Cov

0.0002693593653154059

In [50]:
XLE_Cov=XLE_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'].cov(SPY_daily_returns_df.reset_index(level = 1, drop= True)['daily_returns'])
XLE_Cov

0.00032882729778066575

In [51]:
year_trading_days = 252

In [52]:
# Annualized returns of RTH
RTH_Annual_Ret = RTH_daily_returns_df.mean()*252
RTH_Annual_Ret

daily_returns    0.222666
dtype: float64

In [53]:
# Annualized returns of IYR
IYR_Annual_Ret = IYR_daily_returns_df.mean()*252
IYR_Annual_Ret

daily_returns    0.0986
dtype: float64

In [54]:
# Annualized returns of XLE
XLE_Annual_Ret = XLE_daily_returns_df.mean()*252
XLE_Annual_Ret

daily_returns    0.32279
dtype: float64

In [55]:
# Annualized std dev of RTH
annual_std_dev_RTH = RTH_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_RTH

daily_returns    0.227511
dtype: float64

In [56]:
# Annualized std dev of IYR
annual_std_dev_IYR = IYR_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_IYR

daily_returns    0.305708
dtype: float64

In [57]:
# Annualized std dev of XLE
annual_std_dev_XLE = XLE_daily_returns_df.std() * np.sqrt(year_trading_days)
annual_std_dev_XLE

daily_returns    0.477834
dtype: float64

In [58]:
result = pf.create_returns_tear_sheet(AMZN_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("amzn_prepan_tearsheet.png", format = "png")

Start date,2020-03-03
End date,2022-03-01
Total months,24
,Backtest
Annual return,24.4%
Cumulative returns,54.7%
Annual volatility,34.2%
Sharpe ratio,0.81
Calmar ratio,0.95
Stability,0.45
Max drawdown,-25.6%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'

In [ ]:
result = pf.create_returns_tear_sheet(RTH_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("RTH_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(AMT_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("amt_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(IYR_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("iyr_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(XOM_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("xom_prepan_tearsheet.png", format = "png")

In [ ]:
result = pf.create_returns_tear_sheet(XLE_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("xle_prepan_tearsheet.png", format = "png")


In [ ]:
result = pf.create_returns_tear_sheet(SPY_daily_returns_df.reset_index(level = 1,drop = True).daily_returns,
                                           return_fig=True)
#result.savefig("spy_prepan_tearsheet.png", format = "png")